### Simple Gen AI app using Lnagchain

In [ ]:
import os ##using to set up environmental variable
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true" ## This is required for langchain
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [1]:
# Import beautifulsoup4 library:- It helps to scrap the entire website data

## Data Ingestion -- From the website we need to scrap the data

from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


# WebBaseLoader is something kind of library where what ever link you specifically get or give, it will be able to scrap the entire content from that particular website

pre requisite is we should have beautifulsoup4 library because internally webbaseloader uses beautifulsoup4 

In [ ]:
loader = WebBaseLoader("Website Link")
loader

# Every RAG application performs this thing. First of all we read our entire data source from a specific data source itself. Then we will load this and convert this into our documents. Once we get that document. It is a huge document right. Now in this particular case if it is a huge document, we need to divide this entire document into chunks. You cannot directly give this entire document to our LLM Model. The reason is very simpl with respect to every LLM model, there is some context size. There is a limitation with respect to context size. And as we go ahead and see different different models that are goin to come up, the context size will keep on increasing. But it is always a good idea that we divide this entire document into chunks of text. After we divide all that data into chunks, we convert this into vectors by using some king of vector embedding( Vector embedding are some techniques where  we are able to convert all this text into vectors). After we get all the vector embeddings we try to store it in a vector store DB. So these are he steps.

# Load Data --> Docs --> Divide our text into chunks --> text --> vectors --> Vector Embeddings --> Vector Store DB

In [ ]:
docs = loader.load()
docs

In [ ]:
# Since we have to split our entire documents into chunks of text we will use lanchain_text_splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

# In this we also have an option to specify our chunk size. And I can also overlap my text while doing this splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

# Now we will use this textsplitter and we are going to split all the document in to multiple documents
documents = text_splitter.splti_documents(docs)

In [ ]:
documents

In [ ]:
# Converting text chunks to vectors. The reason we work with Q and A chatbot or document Q&A chatbot, let's consider Rag application, a very simple algorithm is basically used name "COSINE SIMILARITY". And based on the cosine similarity usually gets applied in the vectors itself. 
# Embedding Techniques
from langchain_openai import OpenAIEmbeddings

#OpenAIEmbeddings is a very efficient embedding technique
embeddings= OpenAIEmbeddings()

In [ ]:
# We have to store these vectors into some kind of vector database
# Faiss vector database is created by facebook and you can use it very much efficiently and there also similarity search is applied in the backend
# import faiss-cpu for this

import langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings) # First the documents get embedded into vectors and then get stored in db
# finally you will be able to see that you are having this vector stored db

In [ ]:
vectorstoredb
# you will also be able to save this in your local  environment or in harddisk whereever you want.
# Now based on this vector store db we can query anything an get the response

In [ ]:
# Query from a vector store db 

query="Langsmith has two usage limits: total traces and extended"
result = vectorstoredb.similarity_search(query)

# Once I get the result, I can go ahead and write result of zero and inside that we will be having a key which is called page content.
result[0].page_content


# Querying from vector store db will just try to give you based in the context, all the information that is available near those vectors. It will just come and display in front of you. But, let's say I want to ask a question which is much more meaningful. I really need to provide context with respect to that particular question. So that is the reason why we will be using retrieval chain. This is a very important step because whereever we will specifically be working, we have to use this retriever. We have to use this chain in most of that Rag applications or document Q&A chains.

In [ ]:
from langchain_openai import ChatOpenAI
llm - ChatOpenAI(model="gpt-4.0")

In [ ]:
# Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_ocuments_chain #Create a chain for passing a list of documents to the model(helps in providing the context to the model increasing the spead of search).
# MyLLM model should also have a contect about any documents to answer it muh more properly.

from langchain_core.prompts import ChatPromptTemplate
# from this chatprompttemplate I am going to give my own custom prompt

prompt= ChatPromptTemplate.from_template(
    """
Answer the following question based on the provided context::
<context>
{context}
</context>
    """

)
#Internally I have created a key on the particular name "context". This context is the information I will be giving to my LLM model regarding the information about the documents or text

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

# document_chain is a runnable binding and the mapper is somethin like this 
# first chat prompt template, then chat open AI then string output parser. All has been combined together in the form of a chain. After going through all this chain it should be able to give me some context information about the thing I am searching for right.

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Langsmith has two usage limits: total traces and extended traces" # This is the input which the user is basically giving 
    "context":[Document(page_content="Langsmith has two usage limits: total traces and extended")]# this is the context which I really need to give
})

However we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most rlevant documents and pass those in for a given question.

In [ ]:
### Retriever 

vectorstoredb ## This is having all the vectors information available

Retriever can be considered as an interface and it's responsibiity is that if anybody asks any input, then this interface will just be a way of probably getting the data from the vector store DB where we don't even need to do the similarity search. So after we create this particular vector store db, we convet tis vector store db into a retriever. Retriever with respect to any input, It will be able to pass that particular input through this retriever and get the response from the vector store db. You can onsider this as a pathway to probably get the information from vector store db. 

In [ ]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retriever_chain
retrieval_chain=create_retriever_chain(retriever,document_chain)

In [ ]:
retrieval_chain

In [ ]:
## Get  the response from the LLM

response=retrival_chain.invoke({}"input":"Langsmith has two usage limits: total traces and extended})
response['answer']

In [ ]:
response

In [ ]:
respnse['context']